# Question 3 - Assignment 1

The third task is to provide the sentiment analysis of the text obtained from the Rotten Tomatoes dataset. The sentiments can be classified into 5 categories.

The task is achieved using a 3-layer LSTM with a fully connected layer to give the 5 outputs. Cross-Entropy Loss function is used along with the Stochastic Gradient Descent optimizer.

The code has been written using the PyTorch Deep Learning Framework.

### Imports

In [0]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from collections import Counter
import numpy as np

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F

### LSTM Class Definition for Sentiment Analysis

In [0]:
class LSTM(nn.Module):

    def __init__(self, vocabulary_size, output_size, embedding_dimension, hidden_dimension, number_layers, drop_prob=0.5):
        
        super().__init__()

        self.output_size = output_size
        self.number_layers = number_layers
        self.hidden_dimension = hidden_dimension
        
        self.embedding = nn.Embedding(vocabulary_size, embedding_dimension)
        self.lstm = nn.LSTM(embedding_dimension, hidden_dimension, number_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(0.3)
        
        self.fc = nn.Linear(hidden_dimension, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden_state):
        
        batch_size = x.size(0)

        embeddings = self.embedding(x)
        lstm_output, hidden_state = self.lstm(embeddings, hidden_state)
        
        lstm_output = lstm_output[:,-1,:]
        output = self.dropout(lstm_output)
        output = self.fc(output)
        sigmoid_output = self.sig(output)
        
        sigmoid_output = sigmoid_output.view(batch_size, -1)
        
        return sigmoid_output, hidden_state
    
    
    def init_hidden(self, batch_size):
        
        weight = next(self.parameters()).data
        
        if gpu:
            hidden_state = (weight.new(self.number_layers, batch_size, self.hidden_dimension).zero_().cuda(),
                  weight.new(self.number_layers, batch_size, self.hidden_dimension).zero_().cuda())
        else:
            hidden_state = (weight.new(self.number_layers, batch_size, self.hidden_dimension).zero_(),
                      weight.new(self.number_layers, batch_size, self.hidden_dimension).zero_())
        
        return hidden_state

### Helper Functions for Text

In [0]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
   
    text = str(text).lower()
    text = BeautifulSoup(text, "lxml").text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

def padding_features(string_int, sequence_length=100):
    
    features = np.zeros((len(string_int), sequence_length), dtype = int)
    
    for i, string in enumerate(string_int):
        
        string_len = len(string)
        
        if string_len <= sequence_length:
            
            zeroes = list(np.zeros(sequence_length-string_len))
            new = zeroes+string
        
        elif string_len > sequence_length:
            
            new = string[0:sequence_length]
        
        features[i,:] = np.array(new)
    
    return features

### Read Data & Processing

In [0]:
data = pd.read_table('mrdata.tsv')
data['Phrase'] = data['Phrase'].apply(clean_text)
del data['PhraseId']
del data['SentenceId']
data.head(10)

### Encoded Text to Integers

In [0]:
all_text = ' '.join(data['Phrase'])
words = all_text.split()
count_words = Counter(words)
total_words = len(words)
sorted_words = count_words.most_common(total_words)

In [0]:
vocabulary_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}
string_int = []

In [0]:
for string in data['Phrase']:
    
    r = [vocabulary_to_int[w] for w in string.split()]
    string_int.append(r)

### Encode Labels

In [0]:
encoded_labels = np.array([int(i) for i in data['Sentiment']])

### Truncate & Process Data

In [0]:
string_length = [len(x) for x in string_int]
string_int = [string_int[i] for i, l in enumerate(string_length) if l>0]
encoded_labels = [encoded_labels[i] for i, l in enumerate(string_length) if l>0]
features = padding_features(string_int)

### Split Data into Train & Test Datasets

In [0]:
split_frac = 0.8
len_feat = len(features)
train_x = np.array(features[0:int(split_frac*len_feat)])
train_y = np.array(encoded_labels[0:int(split_frac*len_feat)])
test_x = np.array(features[int(split_frac*len_feat):])
test_y = np.array(encoded_labels[int(split_frac*len_feat):])

In [0]:
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

batch_size = 1024

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

### Variable Declaration

In [0]:
vocabulary_size = len(vocabulary_to_int)+1
output_size = 5
embedding_dimension = 400
hidden_dimension = 256
number_layers = 3
gpu = False
lr=0.001
net = LSTM(vocabulary_size, output_size, embedding_dimension, hidden_dimension, number_layers)

### Loss Function and Optimizer

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=lr)

epochs = 10

counter = 0
clip = 5

if gpu:
  net.cuda()

net.train()

### Train the Network

In [0]:
for e in range(epochs):
    
    hidden_state = net.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        
        counter += 1

        if gpu:
          inputs, labels = inputs.cuda(), labels.cuda()

        hidden_state = tuple([each.data for each in hidden_state])

        net.zero_grad()

        inputs = inputs.type(torch.LongTensor)
        
        if((inputs.shape[0],inputs.shape[1]) != (batch_size,100)):
          
          print('Input Shape Issue:', inputs.shape)
          continue
        
        output, hidden_state = net(inputs, hidden_state)
        
        loss = criterion(output.squeeze(), labels)
        loss.backward()

        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        print("Epoch: {}/{}...".format(e+1, epochs), "Step: {}...".format(counter), "Loss: {:.6f}...".format(loss.item()))

### Compute Test Results

In [0]:
test_losses = []
num_correct = 0

hidden_state = net.init_hidden(batch_size)

net.eval()

for inputs, labels in test_loader:

    hidden_state = tuple([each.data for each in hidden_state])

    if gpu:
      inputs, labels = inputs.cuda(), labels.cuda()
    
    inputs = inputs.type(torch.LongTensor)
    
    if((inputs.shape[0],inputs.shape[1]) != (batch_size,100)):
          
          print('Input Shape Issue:', inputs.shape)
          continue
    
    output, hidden_state = net(inputs, hidden_state)
    
    test_loss = criterion(output.squeeze(), labels)
    test_losses.append(test_loss.item())
    
    out = F.softmax(output)
    out = out.max(dim=1)[1]
    
    correct_tensor = out.eq(labels)
    correct = np.squeeze(correct_tensor.numpy()) if not gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


print("Test Loss: {:.3f}".format(np.mean(test_losses)))

test_accuracy = num_correct/len(test_loader.dataset)
print("Test Accuracy: {:.3f}".format(test_accuracy))